# CHAPTER 7. Market Basket Analysis

### 연관성분석(Association Analysis)

- 연관성 규칙을 통해 하나의 거래나 사건에 포함되어 있는 둘 이상의 품목 간 상호 연관성을 발견해 내는 것.

###  연관규칙의 현장 적용사례
- 어떤 상품들이 함께 잘 팔리나?
-  제품을 구입한 고객에게 함께 팔 수 있는 것은?
    - ** 상품 진열에 영향 **


- 연관 규칙을 발견해 내는 작업이란, 그 데이터 안에 있는 항목 간의
종속관계를 찾아 내는 작업
- 마케팅에서는 고객의 장바구니 안에 들어있는 품목간의 관계를 알아본다는 의미에서 장바구니 분석이라고 함
- 연관분석 : 동시에 구매 될 가능성이 큰 상품을 찾아 내어 매출의 증대에 기여하는 분석
- 서점 : 특정 책을 구입한 고객이 어떤 책을 구입했는지, 어떤 책에 관심이 있는지 연결고리를 만듦
- 신용카드 : 수 많은 매출 데이터를 표본으로 도난 카드의 사용 유형 패턴 발견

###  연관규칙의 예
-  금요일, 일반 소매점 고객들이 일회용 기저귀와 캔맥주를 함께 구입
![](07_01.jpg)


###  연관규칙 해석
- 일주일간 쓸 물건을 금요일에 구입하는데 젊은 부부들은 아이를 위해 기저귀를 구매하고, TV 앞에서 마실 맥주도 구입한다는 것이 연관규칙에 의해 얻어짐
- 이러한 정보를 통해 상품을 진열 할 때 기저귀 옆에 맥주, 유아용품 또는 감자칩 등을 진열하여 매출을 증가 시킬 수 있는 매장의 레이아웃이 됨

### 연관규칙 평가
 - 지지율( Support ) 
 - 신뢰도( Confidence ) 
 - 향상도( Lift, Improvement )

### 지지율( Support ) 

![](07_02.jpg)

![](07_03.jpg)

####  지지율의 단점
- 표본수가 적은 경우
    - 연관관계에 대한 통계적 유의성을 증명하기 어려움
    - 투자한 시간, 비용에 비해 판매량을 증가시키는 기여도가 작음
    
#### 지지율의 목적
- 자주 발생하지 않는 규칙을 제거하는데 사용

### 신뢰도( Confidence ) 

![](07_04.jpg)

### 향상도( Lift, Improvement )

![](07_05.jpg)

## Market Basket Analysis Using MapReduce

- 품목이 많고, 거래가 많은 상점의 연관분석할때는 계산이 기하급수적으로 늘어나서, 병렬처리 방식이 필요함.

![](07_06.jpg)

- 위와 같이 A, B, C... 상품의 빈도, ( A, B) , ( A, C)상품의 빈도를 구하면, 이를 이용해서 지지율, 신뢰도, 향상도를 쉽게 구할 수 있음.